# Crossref pipeline

We have a simple task, we want to retireve a list of papers that is close to the universe of papers for a particular search string (using the paper title). We are going to use Crossref data and use it to query and get the publication DOIs and download the corresponding PDFs when is possible (we're doing low-key a crime here by using SciHub as a source of PDFs, but more on that later). 

In [1]:
import json
import duckdb
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# from scidownl import scihub_download

/home/topcat/miniforge3/envs/lit_review/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
con = duckdb.connect("../crossref_database.db")

In [88]:
con.sql(
    """
select * from adaptation_mortality_embed_abstracts limit 10;
"""
)

┌────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [96]:
con.execute(
    """
    INSTALL vss; 
    LOAD vss; 
    SET GLOBAL hnsw_enable_experimental_persistence = true;
    """
)

con.execute(
    """
    DROP INDEX IF EXISTS my_hnsw_index;
    CREATE INDEX my_hnsw_index ON adaptation_mortality_embed_abstracts USING HNSW (embeddings);
    """
)

In [97]:
con.sql(
    """
select * from adaptation_mortality_embed_abstracts limit 10;
"""
)

┌────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Check QA

In [105]:
embedding_model_type = "all-mpnet-base-v1"
embedding_model = SentenceTransformer(embedding_model_type)
embeddings = embedding_model.encode(
    "Causal or experimental evidence of adaptation to temperature mortality",
    show_progress_bar=True,
)

embed_size = embeddings.shape[0]

Batches: 100%|██████████| 1/1 [00:00<00:00, 90.65it/s]


In [120]:
from duckdb.typing import VARCHAR


def embed(sentence: str) -> np.ndarray:
    return embedding_model.encode(sentence)


con.create_function("embed_qa", embed, [VARCHAR], "FLOAT[768]")

In [141]:
def search(q: str):
    return con.execute(
        """
        from adaptation_mortality_embed_abstracts as e 
        join adaptation_mortality as m 
        on (m.uuid = e.id) 
        select 
        m.title, 
        m.first_author, 
        m.year, 
        array_cosine_similarity(e.embeddings, embed_qa($q)) as cos_sim, 
        order by similarity desc 
        limit 20;
        """,
        {"q": q},
    ).df()

In [144]:
search(
    "Causal or experimental evidence of adaptation to temperature mortality in the US"
)

,title,first_author,year,similarity
0,Convergence in Adaptation to Climate Change: E...,"Barreca, Alan",2015,0.790013
1,Future Temperature‐Related Deaths in the U.S.:...,"Lee, Jangho",2023,0.778201
2,Spatial and intraseasonal variation in changin...,"Spangler, Keith R.",2021,0.728014
3,Winter mortality in a warming climate: a reass...,"Ebi, Kristie L.",2013,0.722912
4,"Climate Change, Mortality, and Adaptation: Evi...","Deschênes, Olivier",2011,0.706474
5,The deadly effect of day-to-day temperature va...,"Hovdahl, Isabel",2022,0.700806
6,Summer temperature variability and long-term s...,"Zanobetti, Antonella",2012,0.690108
7,The mortality burden of extreme heat in Connec...,"Goddard, Emily",2023,0.685727
8,Heat Adaptation among the Elderly in Spain (19...,"Navas-Martín, Miguel Ángel",2023,0.685694
9,"Extreme temperatures, mortality, and adaptatio...","Liao, Hua",2023,0.675358


### Testing Tor

In [1]:
from adaptation_reviewer.download import get_current_ip, renew_tor_ip

renew_tor_ip(password="torpasslitreview")

In [2]:
get_current_ip()

'{\n  "origin": "45.141.215.17"\n}\n'

In [159]:
import requests


def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {
        "http": "socks5://127.0.0.1:9050",
        "https": "socks5://127.0.0.1:9050",
    }
    return session


# Make a request through the Tor connection
# IP visible through Tor
session = get_tor_session()
print(session.get("http://httpbin.org/ip").text)
# Above should print an IP different than your public IP

# Following prints your normal public IP
print(requests.get("http://httpbin.org/ip").text)


{
  "origin": "192.42.116.178"
}

{
  "origin": "171.67.80.243"
}



### Marshall's papers

In [34]:
con.sql(
    """
SELECT 
hash(doi) as id,
doi,
title,
given_1 ||' '||family_1 as name,
year::int as year,
type, link
FROM read_parquet("/mnt/drive/lit_review_data/crossref_parquet_flatten/*.parquet", union_by_name=True)
where given_1 = 'Marshall' AND family_1 = 'Burke'
AND type = 'journal-article'
AND abstract IS NOT NULL
"""
)

┌──────────────────────┬─────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┬───────┬─────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│          id          │                   doi                   │                                               title                                               │      name      │ year  │      type       │                                                                                                                                                                     link                                                                            

In [47]:
import requests


headers = {
    "Host": "pubs-aeaweb-org.stanford.idm.oclc.org",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:133.0) Gecko/20100101 Firefox/133.0",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "If-Modified-Since": "Thu, 19 Dec 2024 18:08:02 GMT",
}

# Donwload pdf in url
url = (
    "https://pubs-aeaweb-org.stanford.idm.oclc.org/doi/pdf/10.1257/pol.20130025"
)

r = requests.get(url)
with open("file.pdf", "wb") as f:
    f.write(r.content)

In [48]:
r.content

b'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background

In [33]:
import pandas as pd

pd.read_parquet(
    "/mnt/drive/lit_review_data/crossref_parquet_flatten/2700_2799.parquet"
).columns

Index(['given_1', 'given_2', 'given_3', 'given_4', 'family_1', 'family_2',
       'family_3', 'family_4', 'affiliation_1', 'affiliation_2',
       'affiliation_3', 'affiliation_4', 'doi', 'title', 'url', 'publisher',
       'is_referenced_by_count', 'container_title', 'reference_count', 'type',
       'volume', 'language', 'author', 'link', 'abstract',
       'published_date_parts', 'created_date_parts', 'created_date_time',
       'created_timestamp', 'journal_issue_issue',
       'journal_issue_published_online_date_parts', 'journal_issue',
       'published', 'journal_issue_published_print_date_parts', 'year'],
      dtype='object')

### How many papers in adaptation we have in the Crossref database by year?

In [2]:
duckdb.query(
    """
    SELECT title, 
    type, 
    url, 
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true)
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change')  
    AND type = 'journal-article' order by 4;
    """
)

┌────────────────────────────────────────┬─────────────────┬───────────────────────────────────────────────────┬───────┐
│                 title                  │      type       │                        url                        │ year  │
│                varchar                 │     varchar     │                      varchar                      │ int64 │
├────────────────────────────────────────┼─────────────────┼───────────────────────────────────────────────────┼───────┤
│ Cattle king a climate adaptation pio…  │ journal-article │ http://dx.doi.org/10.1071/ec12228                 │  1974 │
│ The scope for adaptation to climate …  │ journal-article │ http://dx.doi.org/10.1016/s0959-3780(98)00004-1   │  1998 │
│ Risks, opportunities and adaptation …  │ journal-article │ http://dx.doi.org/10.3354/cr011085                │  1998 │
│ Technology innovation as a strategy …  │ journal-article │ http://dx.doi.org/10.1016/s0143-6228(01)00004-2   │  2001 │
│ Book Review: Vulnerability and

In [3]:
# Try to get all the stuff we need!
adapt_papers = duckdb.query(
    """
    SELECT title, 
    container_title,
    doi, 
    url,
    family_1,
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true)
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change')  
    AND type = 'journal-article' order by 4;
    """
).to_df()

adapt_papers.head()

,title,container_title,doi,url,family_1,year
0,Human adaptation to climate change: An introdu...,American Journal of Human Biology,10.1002/ajhb.23530,http://dx.doi.org/10.1002/ajhb.23530,Pisor,2020
1,Beyond skeletal studies: A computational analy...,American Journal of Biological Anthropology,10.1002/ajpa.24932,http://dx.doi.org/10.1002/ajpa.24932,Bastir,2024
2,The spatial representation of business models ...,Business Strategy &amp; Development,10.1002/bsd2.92,http://dx.doi.org/10.1002/bsd2.92,DiBella,2019
3,Business adaptation to climate change: America...,Business Strategy and the Environment,10.1002/bse.2316,http://dx.doi.org/10.1002/bse.2316,Rivera,2019
4,A relational view of climate adaptation in the...,Business Strategy and the Environment,10.1002/bse.2375,http://dx.doi.org/10.1002/bse.2375,Canevari‐Luzardo,2019


In [9]:
duckdb.query(
    """
    SELECT 
    title, 
    type, 
    url, 
    abstract, 
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true) 
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change') AND 
    type = 'journal-article' and abstract IS NOT NULL
    ORDER by 5;
    """
)

┌──────────────────────┬─────────────────┬──────────────────────┬──────────────────────────────────────────────┬───────┐
│        title         │      type       │         url          │                   abstract                   │ year  │
│       varchar        │     varchar     │       varchar        │                   varchar                    │ int64 │
├──────────────────────┼─────────────────┼──────────────────────┼──────────────────────────────────────────────┼───────┤
│ Reducing hazard vu…  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>O…  │  2006 │
│ Community‐based di…  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>C…  │  2006 │
│ Local initiatives …  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>C…  │  2006 │
│ Modeling agricultu…  │ journal-article │ http://dx.doi.org/…  │ <jats:p>An approach that integrates biophy…  │  2008 │
│ A multi-institutio…  │ journal

In [12]:
duckdb.query(
    """
    with clean as (
        SELECT list_extract(affiliation_1, 1) as first_author_affiliation,
        list_extract(list_extract(published['date-parts'], 1),1) as year
        FROM read_parquet('/mnt/drive/crossref_april_parquet_normalized/*.parquet', union_by_name=true)
        WHERE affiliation_1 IS NOT NULL
        ) SELECT first_author_affiliation, count(*) AS count 
        FROM clean 
        WHERE first_author_affiliation IS NOT NULL and year == 2020
        GROUP BY first_author_affiliation 
        ORDER by count desc;
    """
)

┌─────────────────────────────────────────────────────────────────────────────────────────┬───────┐
│                                first_author_affiliation                                 │ count │
│                                         varchar                                         │ int64 │
├─────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ University of Washington                                                                │ 37393 │
│ Department of Chemistry                                                                 │  3678 │
│ California Institute of Technology                                                      │  1110 │
│ University of Massachusets Medical School                                               │  1075 │
│ Los Alamos National Lab. (LANL), Los Alamos, NM (United States)                         │  1003 │
│ Stanford University                                                                     │   828 │


In [25]:
duckdb.query(
    """
    SELECT container_title, title, type, url, list_extract(list_extract(published['date-parts'], 1),1) as year
    FROM read_parquet('/mnt/drive/crossref_april_parquet_normalized/*.parquet', union_by_name=true)
    WHERE contains(container_title, 'development')
    and list_extract(list_extract(published['date-parts'], 1),1) == 2020
    and type = 'journal-article'
    """
)

┌──────────────────────┬──────────────────────┬─────────────────┬──────────────────────────────────────────────┬───────┐
│   container_title    │        title         │      type       │                     url                      │ year  │
│       varchar        │       varchar        │     varchar     │                   varchar                    │ int64 │
├──────────────────────┼──────────────────────┼─────────────────┼──────────────────────────────────────────────┼───────┤
│ Advances in Neurod…  │ Building Social Sk…  │ journal-article │ http://dx.doi.org/10.1007/s41252-020-00149-8 │  2020 │
│ Journal of Neurode…  │ A telehealth appro…  │ journal-article │ http://dx.doi.org/10.1186/s11689-019-9302-0  │  2020 │
│ Journal of Neurode…  │ Health comorbiditi…  │ journal-article │ http://dx.doi.org/10.1186/s11689-019-9306-9  │  2020 │
│ Advances in Neurod…  │ Mindfulness Traini…  │ journal-article │ http://dx.doi.org/10.1007/s41252-020-00148-9 │  2020 │
│ South of Russia: e…  │ Ecologi

## Semantic Scholar Tests

In [8]:
import requests
from pprint import pprint

base_url = "https://api.semanticscholar.org/datasets/v1/release/"

# Set the release id
release_id = "2024-12-24"

# Make a request to get datasets available the latest release
response = requests.get(base_url + release_id)

# Print the response data
pprint(response.json())

{'README': 'Semantic Scholar Academic Graph Datasets\n'
           '\n'
           'These datasets provide a variety of information about research '
           'papers taken from a snapshot in time of the Semantic Scholar '
           'corpus.\n'
           '\n'
           'This site is provided by The Allen Institute for Artificial '
           'Intelligence (“AI2”) as a service to the\n'
           'research community. The site is covered by AI2 Terms of Use and '
           'Privacy Policy. AI2 does not claim\n'
           'ownership of any materials on this site unless specifically '
           'identified. AI2 does not exercise editorial\n'
           'control over the contents of this site. AI2 respects the '
           'intellectual property rights of others. If\n'
           'you believe your copyright or trademark is being infringed by '
           'something on this site, please follow\n'
           'the "DMCA Notice" process set out in the Terms of Use '
           '(https:/

In [9]:
response.json()

{'release_id': '2024-12-24',
 'README': 'Semantic Scholar Academic Graph Datasets\n\nThese datasets provide a variety of information about research papers taken from a snapshot in time of the Semantic Scholar corpus.\n\nThis site is provided by The Allen Institute for Artificial Intelligence (“AI2”) as a service to the\nresearch community. The site is covered by AI2 Terms of Use and Privacy Policy. AI2 does not claim\nownership of any materials on this site unless specifically identified. AI2 does not exercise editorial\ncontrol over the contents of this site. AI2 respects the intellectual property rights of others. If\nyou believe your copyright or trademark is being infringed by something on this site, please follow\nthe "DMCA Notice" process set out in the Terms of Use (https://allenai.org/terms).\n\nSAMPLE DATA ACCESS\nSample data files can be downloaded with the following UNIX command:\n\nfor f in $(curl https://s3-us-west-2.amazonaws.com/ai2-s2ag/samples/MANIFEST.txt)\n  do curl 

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
import requests
import os

base_url = "https://api.semanticscholar.org/datasets/v1/release/"

# This endpoint requires authentication via api key
api_key = os.getenv("S2_API_KEY")
headers = {"x-api-key": api_key}

# Set the release id
release_id = "2024-12-24"

# Define dataset name you want to download
dataset_name = "abstracts"

# Send the GET request and store the response in a variable
response = requests.get(
    base_url + release_id + "/dataset/" + dataset_name, headers=headers
)

# Process and print the response data
print(response.json())

{'message': 'Forbidden'}


In [ ]:
import requests
import json

# Specify the search term
query = '"generative ai"'

# Define the API endpoint URL
url = "http://api.semanticscholar.org/graph/v1/paper/search/bulk"

# Define the query parameters
query_params = {
    "query": '"generative ai"',
    "fields": "title,url,publicationTypes,publicationDate,openAccessPdf",
    "year": "2023-",
}


# Define headers with API key
headers = {"x-api-key": api_key}

# Send the API request
response = requests.get(url, params=query_params, headers=headers).json()